In [ ]:
!pip install azure-storage-blob
!pip install requests
!pip install beautifulsoup4 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 392.2/392.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.7 MB/s eta 0:00:00


In [ ]:
import requests

def check_connectivity(url):
  try:
    response = requests.get(url, timeout=5)  # Set a timeout to avoid waiting too long
    response.raise_for_status()  # Raise an exception for bad status codes
    print(f"{url} is reachable!")
  except requests.exceptions.RequestException as e:
    print(f"{url} is not reachable. Error: {e}")

check_connectivity('http://repository.agrosavia.co')

http://repository.agrosavia.co is reachable!


In [ ]:
from azure.storage.blob import BlobServiceClient

# Configuración de Azure Blob Storage
account_url = "https://storageyocampo.blob.core.windows.net"
credential = "sv=2022-11-02&ss=b&srt=sco&sp=rwdlaciytfx&se=2025-01-01T01:34:15Z&st=2024-07-14T17:34:15Z&spr=https,http&sig=bM3SKGXXXXXXXXXXXUXX3D"
#***** se especifica el nombre del contenedor objetivo
# Quitar el comentario si se trata de otro Blob Container obtenido en otra ejecución.
container_name ="cacao20240714183705"

folder_name = "pdf/"


# Crear el cliente de servicio de blob
blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
container_client = blob_service_client.get_container_client(container_name)

# Listar archivos en la carpeta de video
print(f"Archivos en la ruta {container_name}/{folder_name}:")
blob_list = container_client.list_blobs(name_starts_with=folder_name)
for blob in blob_list:
    print(blob.name)

Archivos en la ruta cacao20240714183705/pdf/:
pdf/109429_67478.pdf
pdf/109691_67388.pdf
pdf/109927_4359.pdf
pdf/109933_4392.pdf
pdf/109936_4393.pdf
pdf/109939_4394.pdf
pdf/109942_10147.pdf
pdf/109944_10148.pdf
pdf/109948_11344.pdf
pdf/109951_11345.pdf
pdf/109954_11346.pdf
pdf/109957_11347.pdf
pdf/109960_11348.pdf
pdf/109969_11351.pdf
pdf/109972_11352.pdf
pdf/109975_11353.pdf
pdf/109984_11357.pdf
pdf/109987_11358.pdf
pdf/109990_11359.pdf
pdf/110001_11615.pdf
pdf/110004_11616.pdf
pdf/110017_12399.pdf
pdf/110020_13599.pdf
pdf/110024_11354.pdf
pdf/110626_55408.pdf
pdf/110755_67942.pdf
pdf/110791_67985.pdf
pdf/110819_55416.pdf
pdf/110822_55417.pdf
pdf/110825_55418.pdf
pdf/110828_55419.pdf
pdf/110831_55420.pdf
pdf/110834_55421.pdf
pdf/110846_67995.pdf
pdf/13600.pdf
pdf/17567.pdf
pdf/17597.pdf
pdf/20844_1.pdf
pdf/20909_109.pdf
pdf/20922_124.pdf
pdf/21431_1354.pdf
pdf/21788_1906.pdf
pdf/21933_2256.pdf
pdf/22320_2985.pdf
pdf/22453_3226.pdf
pdf/22498_3275.pdf
pdf/22544_1.pdf
pdf/22712_3668.pdf
p

Este script:

1.   Se conecta al contenedor de Azure Blob Storage y lista los archivos PDF en la carpeta especificada.

2.   Descarga cada archivo PDF temporalmente.

3.   Usa la librería PyMuPDF (importada como fitz) para verificar si el PDF contiene texto extraíble.

4.   Crea una tabla con el nombre del archivo, su peso en Kbytes y su tipo.

5.   Guarda los resultados en un archivo CSV temporalmente.

6.   Sube el archivo CSV al Blob Storage en la raíz del contenedor especificado con el nombre resultado_detectar_type_pdf.csv.

In [ ]:
!pip install PyMuPDF
!pip install azure-storage-blob PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 63.3 MB/s eta 0:00:00


In [ ]:
from azure.storage.blob import BlobServiceClient
import fitz  # PyMuPDF
import pandas as pd

# Configuración de Azure Blob Storage
account_url = "https://storageyocampo.blob.core.windows.net"
credential = "sv=2022-11-02&ss=b&srt=sco&sp=rwdlaciytfx&se=2025-01-01T01:34:15Z&st=2024-07-14T17:34:15Z&spr=https,http&sig=bM3SKbM3SKGXXXXXXXXXXXU3D"
container_name = "cacao20240714183705"
folder_name = "pdf/"

# Crear el cliente de servicio de blob
blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
container_client = blob_service_client.get_container_client(container_name)

# Listar archivos en la carpeta de PDF
blob_list = container_client.list_blobs(name_starts_with=folder_name)

# Función para verificar si el PDF es de texto extraíble
def is_text_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            text = doc[page_num].get_text()
            if text.strip():
                return True
        return False
    except Exception as e:
        print(f"Error al procesar {pdf_path}: {e}")
        return False

# Crear una lista para almacenar la información de los archivos
files_data = []

for blob in blob_list:
    blob_client = container_client.get_blob_client(blob.name)

    # Descargar el archivo PDF temporalmente
    download_file_path = "/tmp/" + blob.name.split('/')[-1]
    with open(download_file_path, "wb") as download_file:
        download_file.write(blob_client.download_blob().readall())

    # Verificar si el PDF es de texto extraíble
    is_text = is_text_pdf(download_file_path)

    # Añadir la información del archivo a la lista
    file_info = {
        "nombre_archivo": blob.name,
        "peso_kbytes": blob.size / 1024,
        "tipo": 1 if is_text else 0
    }
    files_data.append(file_info)

    # Mostrar el resultado del análisis
    print(f"Archivo: {file_info['nombre_archivo']}, Peso: {file_info['peso_kbytes']:.2f} KB, Tipo: {'Texto' if file_info['tipo'] == 1 else 'Imagen'}")

# Crear un DataFrame con los resultados
df = pd.DataFrame(files_data)

# Guardar la tabla en un archivo CSV temporalmente
output_csv_path = '/tmp/resultado_detectar_type_pdf.csv'
df.to_csv(output_csv_path, index=False)

# Subir el archivo CSV al Blob Storage
output_blob_name = "resultado_detectar_type_pdf.csv"
blob_client = container_client.get_blob_client(output_blob_name)

with open(output_csv_path, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print(f"\nArchivo {output_blob_name} subido a Azure Blob Storage en la ruta / del contenedor {container_name}")


Archivo: pdf/109429_67478.pdf, Peso: 5734.25 KB, Tipo: Texto
Archivo: pdf/109691_67388.pdf, Peso: 6010.64 KB, Tipo: Texto
Archivo: pdf/109927_4359.pdf, Peso: 770.78 KB, Tipo: Texto
Archivo: pdf/109933_4392.pdf, Peso: 1126.49 KB, Tipo: Texto
Archivo: pdf/109936_4393.pdf, Peso: 287.91 KB, Tipo: Texto
Archivo: pdf/109939_4394.pdf, Peso: 187.50 KB, Tipo: Texto
Archivo: pdf/109942_10147.pdf, Peso: 363.28 KB, Tipo: Imagen
Archivo: pdf/109944_10148.pdf, Peso: 1023.95 KB, Tipo: Texto
Archivo: pdf/109948_11344.pdf, Peso: 286.97 KB, Tipo: Texto
Archivo: pdf/109951_11345.pdf, Peso: 968.12 KB, Tipo: Texto
Archivo: pdf/109954_11346.pdf, Peso: 1590.76 KB, Tipo: Texto
Archivo: pdf/109957_11347.pdf, Peso: 159.06 KB, Tipo: Texto
Archivo: pdf/109960_11348.pdf, Peso: 2442.12 KB, Tipo: Texto
Archivo: pdf/109969_11351.pdf, Peso: 382.58 KB, Tipo: Texto
Archivo: pdf/109972_11352.pdf, Peso: 906.61 KB, Tipo: Texto
Archivo: pdf/109975_11353.pdf, Peso: 407.95 KB, Tipo: Texto
Archivo: pdf/109984_11357.pdf, Peso: 

In [ ]:
import pandas as pd

# Cargar el archivo CSV
resultado_csv_path = '/tmp/resultado_detectar_type_pdf.csv'  # Reemplaza con la ruta correcta
df_resultado = pd.read_csv(resultado_csv_path)

# Contar la cantidad de PDFs de texto y de imagen
conteo_tipos = df_resultado['tipo'].value_counts()

# Obtener la cantidad de PDFs de texto y de imagen
cantidad_texto = conteo_tipos.get(1, 0)
cantidad_imagen = conteo_tipos.get(0, 0)

print(f"Cantidad de PDFs de texto: {cantidad_texto}")
print(f"Cantidad de PDFs de imagen: {cantidad_imagen}")


Cantidad de PDFs de texto: 848
Cantidad de PDFs de imagen: 113
